In [4]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [5]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=4000)

In [7]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings(
    array_size=1000,
    gains_pct_filter=0,
    total_trade_filter=40,
    upside_filter=0,
)

# backtest_settings = BacktestSettings()

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([2, 3, 4, 5, 6]) * 3,
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6, 10]),
    sl_based_on_add_pct=np.array([0.01, 0.05, 0.1]),
    sl_based_on_lookback=np.array([20, 40, 60]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.2, 0.5, 1.0, 1.5]),
    trail_sl_when_pct=np.array([0.2, 0.4, 0.8, 1, 2]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=False,
    long_or_short=LongOrShortType.Long,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=ZeroOrEntryType.Nothing,
)

In [12]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 3,600
Total combinations of settings to test: 21,600
Total candles: 4,000
Total candles to test: 86,400,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,24,58.0,21,37,1.445,36.207,0.317,823.418,14.451,1014.451
1,0,89,58.0,21,37,16.476,36.207,0.068,790.043,164.756,1164.756
2,0,98,50.0,17,33,3.598,34.000,0.365,669.780,35.985,1035.985
3,0,108,42.0,19,23,4.748,45.238,0.179,412.020,47.480,1047.480
4,0,167,52.0,24,28,6.834,46.154,0.111,490.829,68.338,1068.338
...,...,...,...,...,...,...,...,...,...,...,...
176,5,702,46.0,24,22,2.391,52.174,0.144,407.064,23.915,1023.915
177,5,827,50.0,25,25,12.194,50.000,0.002,778.358,121.945,1121.945
178,5,887,44.0,23,21,1.003,52.273,0.110,539.647,10.032,1010.032
179,5,909,46.0,11,35,50.936,23.913,0.356,1485.633,509.364,1509.364


In [9]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
65,2,29,42.0,18,24,65.148,42.857,0.506,608.324,651.479,1651.479
119,3,429,42.0,10,32,26.783,23.810,0.500,806.269,267.832,1267.832
68,2,89,42.0,17,25,25.615,40.476,0.465,608.719,256.147,1256.147
78,2,309,42.0,12,30,40.637,28.571,0.461,670.909,406.374,1406.374
114,3,244,42.0,11,31,66.250,26.190,0.454,735.091,662.504,1662.504
...,...,...,...,...,...,...,...,...,...,...,...
137,4,162,44.0,24,20,13.265,54.545,0.001,393.569,132.654,1132.654
23,0,2469,48.0,11,37,25.100,22.917,0.001,1511.458,250.998,1250.998
131,3,2184,50.0,16,34,3.636,32.000,0.001,986.686,36.361,1036.361
24,0,3199,44.0,10,34,33.063,22.727,0.001,1358.159,330.628,1330.628


In [11]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=2,
    dos_index=29,
    plot_results=True,
)

In [ ]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

returned_data = res.json()

print(returned_data)